# Flights API

Primeiro precisamos registrar no site [Amadeus](https://developers.amadeus.com/register) para pegar a chave de acesso da API.

In [ ]:
#!pip install amadeus

### Initial Configuration

Importamos as bibliotecas que iremos utilizar.

In [ ]:
from datetime import date
import requests
from decimal import Decimal
from dateutil.parser import parse as parse_date
import pandas as pd
from tabulate import tabulate

Precisamos configurar a chave de acesso da API. A [documentação](https://developers.amadeus.com/self-service/apis-docs/guides/authorization) tem mais informações.

In [ ]:
client_id = ''
client_secret = ''

In [ ]:
body = {'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret}

In [ ]:
auth_response = requests.post('https://test.api.amadeus.com/v1/security/oauth2/token',
                              data=body)

In [ ]:
token = auth_response.json()['access_token']

### Function definition

In [ ]:
def call_api(url, **params):
    full_url = 'https://test.api.amadeus.com/{url}'.format(url=url)
    response = requests.get(full_url,
                            params=params,
                            headers={'Authorization': 'Bearer {bearer}'.format(bearer=token),
                                     'Content-Type': 'application/vnd.amadeus+json'})
    return response.json()

Para buscar o código do aeroporto podemos consultar o endpoint [/reference-data/locations](https://developers.amadeus.com/self-service/category/air/api-doc/airport-and-city-search/api-reference)

"O código aeroportuário IATA é uma sigla composta por três letras, utilizada para designar os aeroportos em todo o mundo." ->  International Air Transport Association

In [ ]:
def get_iata(city, country_code):
    response = call_api('v1/reference-data/locations',
                        keyword=city,
                        countryCode=country_code,
                        subType='AIRPORT')
    #print(response)
    return [result['iataCode'] for result in response['data']]

In [ ]:
get_iata('New York', 'US')

Para consultar o preço da passagem podemos fazer uma requisição para o endpoint [/shopping/flight-offers](https://developers.amadeus.com/self-service/category/air/api-doc/flight-low-fare-search/api-reference)

In [ ]:
def get_prices(origin, destination, date, only_direct, adults):
    response = call_api('v2/shopping/flight-offers',
                        originLocationCode=origin,
                        destinationLocationCode=destination,
                        nonStop='true' if only_direct else 'false',
                        departureDate=date.strftime('%Y-%m-%d'),
                        adults=adults)
    #print(response)
    if 'data' not in response:
        print(response)
        return []
    return [(origin, destination, date,Decimal(offer_item['price']['total']),
             parse_date(offer_item['itineraries'][0]['segments'][0]['departure']['at']),
             parse_date(offer_item['itineraries'][0]['segments'][-1]['arrival']['at']),
             len(offer_item['itineraries'][0]['segments']))
            for offer_item in response['data']]

In [ ]:
get_prices('GIG', 'CDG', date(2020, 10, 1), False, 1)

### Define destinations we want to search

In [ ]:
destinations = pd.DataFrame([('Sydney', 'AU', False, ['SYD']), ('Sydney', 'AU', True, ['SYD']), ('Paris', 'FR', False, ['CDG','ORY']), ('Paris', 'FR', True, ['CDG','ORY'])],
                            columns=['city', 'country_code', 'only_direct', 'iata'])
destinations

Precisamos buscar o código IATA de cada cidade

### Get prices

In [ ]:
for row in range(len(destinations)):
  dest = destinations.iloc[row] # pega a linha de numero 'row' -- está no for

  iata = dest['iata'][0] #string da IATA

  prices_IATA = get_prices('GIG', iata, date(2020, 11 ,8), dest['only_direct'], 1) #pega os preços das passagens usando a função get_prices
  df = pd.DataFrame(dest).T #cria um dataframe para cada iteração do for

  prices = [i[3] for i in prices_IATA] #cria lista de preços
  conections = [i[-1] for i in prices_IATA] #cria lista de conecções

  df = pd.DataFrame(df.values.repeat(len(prices), axis=0), columns=df.columns) #cria novo dataframe

  # adiciona colunas ao novo dataframe:
  df['prices'] = prices
  df['conections'] = conections
  df['departure'] = ['GIG' for i in range(len(prices))]
  df['arrival'] = [iata for i in range(len(prices))]

  # print dos resultados
  print(tabulate(df, headers='keys', tablefmt='psql'))

  print('\n \n \n')